In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH16-Agentic-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Agentic-RAG


# Agentic RAG (feat.LangChain)

** 실습 자료 **

소프트웨어정책연구소(SPRi), [AI Brief 스페셜] AI 에이전트 동향

- 저자: 유재흥(AI정책연구실 책임연구원), 이해수(AI정책연구실 선임연구원)
- 발생: 2024.12.10
- 링크: https://spri.kr/posts/view/23798?code=AI-Brief&s_year=&data_page=1
- 파일명: `SPRi AI Brief_Special_AI Agent_241209_F`

(실습 준비)
- 작업 폴더 하위에 `data` 폴더 생성 
- 링크에서 다운로드 받은 파일을 `data` 폴더에 업로드 

### WEB Search

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [4]:
search = TavilySearchResults(k=6)

In [5]:
search.invoke("하나은행 고객센터 전화번호는 무엇인가요?")

[{'url': 'https://blog.naver.com/assist12060/223556362662',
  'content': '* 1599-1111 은 하나은행 고객센터 대표번호로, ARS 안내를 거쳐야 합니다. * 1588-8000 은 하나은행 ARS 없이 바로 상담원과 연결되는 전용 번호입니다.'},
 {'url': 'https://www.kebhana.com/cont/customer/customer01/customer0101/customer010101/index.jsp',
  'content': '자세한 내용은 하나은행 고객센터 (1599-1111) 또는 하나원큐 [메뉴 > 상품가입 > 내 아이 통장 만들기 서비스]메뉴의 안내를 참고해주시기 바랍니다. ※ 인터넷뱅킹('},
 {'url': 'https://m.kebhana.com/cont/customer/customer02/index.jsp',
  'content': '1599-1115. 개인종합자산관리계좌(ISA). 1599-1463. 하나은행 고객센터. 1599-1111 · 1588-1111. 하나카드 콜센터. 1800-1111 · 고객센터 PC버전 영업점 찾기 · 개인정보'},
 {'url': 'https://www.kebhana.com/cont/customer/index.jsp',
  'content': '고객센터 메인 · 예금 1811-6171 · 펀드 1811-6172 · 대출 1599-2222 · 외환 1811-6174 · 어르신 전용 1899-8833.'},
 {'url': 'https://www.kebhana.com/cont/customer/customer02/index.jsp',
  'content': "안내 · 상담원에게연결. 전화하나로 센터 1599-1111, 1588-1111 (0 · 상담원에게 설명. 서비스 받은 내용전달 · 원격 지원시스템 실행. '원격지원PC상담' 시작하기 클릭 · 원격"}]

## 문서 기반 검색

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.tools.retriever import create_retriever_tool

In [7]:
loader = PyPDFLoader("data/SPRi AI Brief_Special_AI Agent_241209_F.pdf")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

split_docs = loader.load_and_split(text_splitter)
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [8]:
retriever.invoke("Apple Intelligence 관련 내용을 문서에서 찾아주세요")

[Document(metadata={'source': 'data/SPRi AI Brief_Special_AI Agent_241209_F.pdf', 'page': 10}, page_content='부가 가치를 높이는 기능으로 작용하여 기기 판매와 Apple 생태계 에 대한 사용자  충성도를 강화하는 데 기여 \n∙Apple 의 고객 중심 개인정보  보호 전략에 따라 Siri는 사용자 데이터 수집을 최소화하며 , Google 이나 \nAmazon 과 달리 광고 모델을 통해 수익 창출을 하지 않음\n£(최신 동정) 2024년 6월 Apple 은 세계개발자대회 (WWDC) 에서 iPhone, iPad, Mac의 핵심 \n기능으로  강력한 생성 모델을 적용한 개인 인텔리전스 시스템인 Apple Intelligence 를 소개21)\n∙생성형 모델의 힘과 개인 맥락을 결합하여 사용자에게 유용하고 관련성 있는 인텔리전스를 제공 \n∙2024년 10월 28일, Apple 은 Apple Intelligence 를 공식 출시하 였고, 사용자들은 이를 활용하여 \n글쓰기를 다듬고 , 알림, 메일, 메시지를 요약하고 , 보다 자연스럽고 강력한 Siri를 경험하고 , Clean Up을 \n사용하여 이미지에서 방해가 되는 물체를 제거하는 등의 작업을 수행 가능22)  \n∙ CEO 팀 쿡(Tim Cook) 은 Apple Intelligence 가 완전히 새로운 경험과 도구를 제공하며 , 수년간의 AI 및 \n머신러닝 혁신을 바탕으로 Apple 의 생성형 모델을 기기의 핵심에 배치해 개인 인텔리전스를 제공하는 \n동시에 개인정보를 보호하여 사용자의 삶을 풍요롭게 할 수 있는 잠재력이 크다고 언급  \n21) Apple, Introducing Apple Intelligence, the personal intelligence system that puts powerful generative models at the core of \niPhone, iPad, and Mac, 2024.6.10

In [9]:
# 도구 형태로 정의
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="use this tool to search information from the PDF document",
)

## 도구 정의

In [10]:
tools = [search, retriever_tool]

## Agent 정의

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [12]:
# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# tool calling agent 정의
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 정의
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

## Agent 실행 

In [13]:
from langchain_teddynote.messages import AgentStreamParser

In [14]:
# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()

In [15]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "2024년 프로야구 플레이오프 진출한 5개 팀을 검색하여 알려주세요."}
)

for step in result:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2024년 프로야구 플레이오프 진출 팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`



[관찰 내용]
Observation: [{'url': 'https://namu.wiki/w/KBO%20%ED%94%8C%EB%A0%88%EC%9D%B4%EC%98%A4%ED%94%84/2024%EB%85%84', 'content': '... 년 한국시리즈가 마지막이다. 나눔 올스타 팀 중 LG 트윈스가 플레이오프 진출을, KIA 타이거즈가 한국시리즈 직행을 하면서 앞으로의 포스트시즌 경기 결과에 상관'}, {'url': 'https://filmk.tistory.com/entry/%ED%94%84%EB%A1%9C%EC%95%BC%EA%B5%AC-%ED%8F%AC%EC%8A%A4%ED%8A%B8%EC%8B%9C%EC%A6%8C-%EA%B2%BD%EA%B8%B0-%EC%9D%BC%EC%A0%95-%EB%B0%8F-%EA%B2%B0%EA%B3%BC-%EC%B4%9D%EC%A0%95%EB%A6%AC2024', 'content': '4) 한국시리즈 일정 ... 2024 한국 프로야구 플레이오프에서 승리한 팀은 KIA와 맞붙게 되며, 그 승부는 한국시리즈 진출을 결정짓는 중요한 대결이 될 것'}, {'url': 'https://aldalsuin.tistory.com/entry/2024-%ED%94%84%EB%A1%9C%EC%95%BC%EA%B5%AC-%ED%94%8C%EB%A0%88%EC%9D%B4%EC%98%A4%ED%94%84-9%EA%B0%9C-%EA%B5%AC%EB%8B%A8%EC%9D%98-%EC%B9%98%EC%97%B4%ED%95%9C-%EC%9A%B0%EC%8A%B9-%EA%B2%BD%EC%9F%81', 'content': '2024 프로야구 플레이오프에 진출한 팀들의 면면을 

In [16]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "Apple Intelligence 관련된 정보를 문서에서 찾아주세요."}
)

for step in result:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: Apple Intelligence
Log: 
Invoking: `pdf_search` with `{'query': 'Apple Intelligence'}`



[관찰 내용]
Observation: 부가 가치를 높이는 기능으로 작용하여 기기 판매와 Apple 생태계 에 대한 사용자  충성도를 강화하는 데 기여 
∙Apple 의 고객 중심 개인정보  보호 전략에 따라 Siri는 사용자 데이터 수집을 최소화하며 , Google 이나 
Amazon 과 달리 광고 모델을 통해 수익 창출을 하지 않음
£(최신 동정) 2024년 6월 Apple 은 세계개발자대회 (WWDC) 에서 iPhone, iPad, Mac의 핵심 
기능으로  강력한 생성 모델을 적용한 개인 인텔리전스 시스템인 Apple Intelligence 를 소개21)
∙생성형 모델의 힘과 개인 맥락을 결합하여 사용자에게 유용하고 관련성 있는 인텔리전스를 제공 
∙2024년 10월 28일, Apple 은 Apple Intelligence 를 공식 출시하 였고, 사용자들은 이를 활용하여 
글쓰기를 다듬고 , 알림, 메일, 메시지를 요약하고 , 보다 자연스럽고 강력한 Siri를 경험하고 , Clean Up을 
사용하여 이미지에서 방해가 되는 물체를 제거하는 등의 작업을 수행 가능22)  
∙ CEO 팀 쿡(Tim Cook) 은 Apple Intelligence 가 완전히 새로운 경험과 도구를 제공하며 , 수년간의 AI 및 
머신러닝 혁신을 바탕으로 Apple 의 생성형 모델을 기기의 핵심에 배치해 개인 인텔리전스를 제공하는 
동시에 개인정보를 보호하여 사용자의 삶을 풍요롭게 할 수 있는 잠재력이 크다고 언급  
21) Apple, Introducing Apple Intelligence, the personal intelligence system that puts powerful generative models at the core of 
iPhone, iPa

## 이전 대화 기억하기 

- 이전의 대화내용을 기억하기 위해서는 `RunnableWithMessageHistory` 를 사용하여 `AgentExecutor` 를 감싸줌
- [Ref] https://wikidocs.net/254682

In [17]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [18]:
# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    # session_id 가 store에 없는 경우
    if session_ids not in store:
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환

In [19]:
# 채팅 메시지 기록이 추가된 에이전트 생성
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"input": "Apple Intelligence 관련된 정보를 문서에서 찾아주세요."},
    # 세션 아이디는 지금은 하드코딩했지만 나중엔 uuid로 발전
    config={"configurable": {"session_id": "SKALA-001"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: Apple Intelligence
Log: 
Invoking: `pdf_search` with `{'query': 'Apple Intelligence'}`



[관찰 내용]
Observation: 부가 가치를 높이는 기능으로 작용하여 기기 판매와 Apple 생태계 에 대한 사용자  충성도를 강화하는 데 기여 
∙Apple 의 고객 중심 개인정보  보호 전략에 따라 Siri는 사용자 데이터 수집을 최소화하며 , Google 이나 
Amazon 과 달리 광고 모델을 통해 수익 창출을 하지 않음
£(최신 동정) 2024년 6월 Apple 은 세계개발자대회 (WWDC) 에서 iPhone, iPad, Mac의 핵심 
기능으로  강력한 생성 모델을 적용한 개인 인텔리전스 시스템인 Apple Intelligence 를 소개21)
∙생성형 모델의 힘과 개인 맥락을 결합하여 사용자에게 유용하고 관련성 있는 인텔리전스를 제공 
∙2024년 10월 28일, Apple 은 Apple Intelligence 를 공식 출시하 였고, 사용자들은 이를 활용하여 
글쓰기를 다듬고 , 알림, 메일, 메시지를 요약하고 , 보다 자연스럽고 강력한 Siri를 경험하고 , Clean Up을 
사용하여 이미지에서 방해가 되는 물체를 제거하는 등의 작업을 수행 가능22)  
∙ CEO 팀 쿡(Tim Cook) 은 Apple Intelligence 가 완전히 새로운 경험과 도구를 제공하며 , 수년간의 AI 및 
머신러닝 혁신을 바탕으로 Apple 의 생성형 모델을 기기의 핵심에 배치해 개인 인텔리전스를 제공하는 
동시에 개인정보를 보호하여 사용자의 삶을 풍요롭게 할 수 있는 잠재력이 크다고 언급  
21) Apple, Introducing Apple Intelligence, the personal intelligence system that puts powerful generative models at the core of 
iPhone, iPa

In [ ]:
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요."},
    # 세션 ID 동일하게 설정 
    config={"configurable": {"session_id": "SKALA-001"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
Here is the translation of the previous response into English:

1. **Introduction and Features**:
   - Apple introduced its personal intelligence system, Apple Intelligence, at the Worldwide Developers Conference (WWDC) in June 2024. This system operates as a core feature of the iPhone, iPad, and Mac, based on a powerful generative model.
   - Apple Intelligence combines the power of generative models with personal context to provide users with useful and relevant intelligence. Users can utilize it to refine their writing, summarize notifications, emails, and messages, and remove distracting objects from images.

2. **Privacy Protection**:
   - Apple adopts a customer-centric privacy strategy, minimizing user data collection through Siri. Apple Intelligence employs on-device learning, processing user data securely on the local device without sending it to Apple servers.

3. **Integration with Siri**:
   - Apple Intelligence is integrated with Siri, which recognizes voice comman

------
## Pipeline 

위에서 작성된 단계별 코드 취합/정리

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser

In [23]:
file_path = "data/SPRi AI Brief_Special_AI Agent_241209_F.pdf"

In [24]:
########## 1. 도구 정의 ##########

### 1-1. Search 도구 ###
search = TavilySearchResults(k=6)

### 1-2. PDF 문서 검색 도구 (Retriever) ###
loader = PyMuPDFLoader(file_path)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

split_docs = loader.load_and_split(text_splitter)
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

retriever = vector.as_retriever()
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="use this tool to search information from the PDF document",
)

### 1-3. tools 리스트에 도구 목록 추가 ###
tools = [search, retriever_tool]

########## 2. LLM 정의 ##########
llm = ChatOpenAI(model="gpt-4o", temperature=0)

########## 3. Prompt 정의 ##########
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 정의 ##########
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 정의 ##########
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

########## 6. 메모리 추가 ##########
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]


# 채팅 메시지 기록이 추가된 에이전트
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

########## 7. Agent 파서 정의 ##########
agent_stream_parser = AgentStreamParser()

아래의 각 실행 내용은 LangSmith TRACE 통해 상세 확인하기

In [25]:
########## 8. 에이전트 실행 ##########

response = agent_with_chat_history.stream(
    {"input": "AI Agent의 기술적 문제를 문서에서 찾아주세요."},
    config={"configurable": {"session_id": "SKALA-002"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: AI Agent 기술적 문제
Log: 
Invoking: `pdf_search` with `{'query': 'AI Agent 기술적 문제'}`



[관찰 내용]
Observation: £ AI 에이전트는 기술적 혁신과 사회적 변화를 이끄는 주요 요소로 자리잡으며, 기업의 업무와 
개인의 삶에 깊이 통합된 형태로 발전하면서, 의사 결정에 혁신적인 변화를 가져올 가능성이 높음52) 
∙(단기적) AI 에이전트는 향후 몇 년간 고객 지원, 데이터 분석, 대화형 인터페이스 등 상용화 가능성이 높은 
분야에서 빠르게 도입될 것으로 예상
 - AI 에이전트가 고객 서비스 자동화, 맞춤형 추천, 실시간 데이터 분석을 통해 효율성을 높이며, 고객 경험 
강화와 비용 절감의 핵심 도구가 될 것으로 전망됨 
50) CNN, Uber self-driving car test driver pleads guilty to endangerment in pedestrian death case, 2023.7.29
51) Gartner, Gartner Top 10 Strategic Technology Trends for 2025, 2024.10.21
52) World Economic Forum, What is an AI agent and what will they do? Experts explain, 2024.7.24

대해 AI 에이전트(AI agents 또는 agentic systems)라고 지칭
1) Russell, S. J., & Norvig, P.  Artificial Intelligence: A Modern Approach (4th ed.). Pearson. 2021
2) Alan Chan 외, Visibility into AI Agents, 2024

∙AI 에이전트들이 더욱 정교한 상호작용과 맥락 인식을 가능하게 하여 고객 서비스, 의료, 금융 분야에서의 
활용이 증가하고, 다중 에이전트 시스템의 협력이 AI 시장 성장

In [26]:
########## 8. 에이전트 실행 ##########

response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 핵심 내용이 누락되지 않도록 요약해 주세요."},
    config={"configurable": {"session_id": "SKALA-002"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
AI 에이전트의 기술적 문제로는 데이터 편향과 정확성 문제가 있습니다. 데이터가 편향적일 경우 AI의 판단에도 왜곡이 발생할 수 있으며, 이는 잘못된 정보 제공으로 이어질 수 있습니다. 특히 챗봇과 같은 대화형 AI에서는 사용자 인터페이스와 내부 구성요소의 편향이 문제를 악화시킬 수 있습니다.


In [27]:
########## 8. 에이전트 실행 ##########

response = agent_with_chat_history.stream(
    {"input": "2024년 프로야구 플레이오프 진출 5개팀을 검색해서 알려주세요."},
    config={"configurable": {"session_id": "SKALA-002"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2024 프로야구 플레이오프 진출 팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024 프로야구 플레이오프 진출 팀'}`



[관찰 내용]
Observation: [{'url': 'https://namu.wiki/w/KBO%20%ED%94%8C%EB%A0%88%EC%9D%B4%EC%98%A4%ED%94%84/2024%EB%85%84', 'content': '... 한국시리즈가 마지막이다. 나눔 올스타 팀 중 LG 트윈스가 플레이오프 진출을, KIA 타이거즈가 한국시리즈 직행을 하면서 앞으로의 포스트시즌 경기 결과에 상관없이'}, {'url': 'https://namu.wiki/w/2024%20%EC%8B%A0%ED%95%9C%20SOL%20Bank%20KBO%20%ED%8F%AC%EC%8A%A4%ED%8A%B8%EC%8B%9C%EC%A6%8C', 'content': '포스트시즌 진출 팀[11][편집]. 1위 - KIA 타이거즈. 진출 확정 - 9월 6일 VS 키움 승리 ... 오랫동안 KBO 리그에서 활약했던 스타들의 은퇴 소식이 올해도 가을야구 기간에'}, {'url': 'https://ko.wikipedia.org/wiki/2024%EB%85%84_KBO_%ED%8F%AC%EC%8A%A4%ED%8A%B8%EC%8B%9C%EC%A6%8C', 'content': '목차\n\n2024년 KBO 포스트시즌\n\n\n\n2024 신한 SOL Bank KBO 포스트시즌은 2024년 10월 2일부터 10월 28일까지 진행됐다. 이번 포스트시즌 역시 한국시리즈 5차전 장소는 정규시즌 우승팀의 홈구장인 광주기아챔피언스필드에서 치러졌다.\n\n진출팀\n\n2024년 포스트시즌에 진출한 팀은 KIA 타이거즈, 삼성 라이온즈, LG 트윈스, 두산 베어스, kt 위즈다.\n\n대진표 [...] 황재균이 볼넷으로 출루해 무사 

In [28]:
########## 8. 에이전트 실행 ##########

response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 SNS 게시글 형태로 작성하세요."},
    config={"configurable": {"session_id": "SKALA-002"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
2024년 프로야구 플레이오프 진출 팀이 확정되었습니다! 🎉⚾️ 올해의 주인공은 KIA 타이거즈, 삼성 라이온즈, LG 트윈스, 두산 베어스, 그리고 KT 위즈입니다! 모두 응원하는 팀의 선전을 기대해 봅시다! 🙌🔥 #프로야구 #플레이오프 #KIA타이거즈 #삼성라이온즈 #LG트윈스 #두산베어스 #KT위즈


In [29]:
########## 8. 에이전트 실행 ##########

response = agent_with_chat_history.stream(
    {"input": "이전의 답변에 한국 시리즈 일정을 추가해 주세요."},
    config={"configurable": {"session_id": "SKALA-002"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 2024 KBO Korean Series schedule
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024 KBO Korean Series schedule'}`



[관찰 내용]
Observation: [{'url': 'https://en.wikipedia.org/wiki/2024_KBO_League_season', 'content': 'Game | Date | Score | Venue | Time | Duration | Attendance\n1 | October 13, 2024 | LG Twins – 4,Samsung Lions – 10 | Daegu Samsung Lions Park | 14:00 KST | 3:15 | 23,550\n2 | October 15, 2024 | LG Twins – 5,Samsung Lions – 10 | 18:30 KST | 3:20 | 23,550\n3 | October 17, 2024 | Samsung Lions – 0,LG Twins – 1 | Jamsil Baseball Stadium | 18:30 KST | 2:48 | 23,750\n4 | October 19, 2024 | Samsung Lions – 1, LG Twins – 0 | 14:00 KST | 2:54 | 23,750\n2024 Korean Series\n\nFinals – best of 7 series. [...] Game | Date | Score | Venue | Time | Duration | Attendance\n1 | October 5, 2024 | KT Wiz – 3, LG Twins – 2 | Jamsil Baseball Stadium | 14:00 KST | 2:36 | 23,750\n2 | October 6, 2024 | KT Wiz – 2,LG Twins – 7 | 14:00 KST

-----
** End of Documents **